In [ ]:
from darts.models import TiDEModel
from darts.dataprocessing.transformers.scaler import Scaler
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from torch.optim import lr_scheduler

In [ ]:
import sys
import os

# Go up two levels from notebook (Training/MLR) to project root
project_root = os.path.abspath(os.path.join(os.getcwd(), "../.."))
sys.path.append(project_root)

print("Project root added to sys.path:", project_root)
# Ensure the model save directory exists
model_save_path = os.path.join('.')
os.makedirs(model_save_path, exist_ok=True)  # Creates directory if it doesn't exist

Data loading

In [ ]:
import pandas as pd
from Training.Helper.dataPreprocessing import TRAIN_DATA_PATH_1990S, get_untransformed_exog
date_col = 'observation_date'

# Load and format training data (only using PCEPI)
train_df = pd.read_csv(TRAIN_DATA_PATH_1990S)
train_df = get_untransformed_exog(train_df)
# Convert the date column to datetime format
train_df[date_col] = pd.to_datetime(train_df[date_col], format='%m/%Y')

# Set the date column as the index
train_df.set_index(date_col, inplace=True)

In [ ]:
from darts import TimeSeries
target_series = TimeSeries.from_series(train_df['fred_PCEPI'])

# Extract the exogenous variables (all columns except 'fred_PCEPI')
exogenous_variables = train_df.drop(columns=['fred_PCEPI'])
exogenous_series = TimeSeries.from_dataframe(exogenous_variables)

Basic model

In [ ]:
model = TiDEModel(input_chunk_length=24, output_chunk_length=6)

# Fit the model with the target series and exogenous variables
model.fit(series=target_series,past_covariates=exogenous_series)

In [ ]:
pred = model.predict(6)
pred.values()

This is terrible 

Split validation and training then scale

In [ ]:
train, val = target_series.split_after(0.8)
train_exo, val_exo = exogenous_series.split_after(0.8)

In [ ]:
targetScaler = Scaler()  # default uses sklearn's MinMaxScaler
scaled_target = targetScaler.fit_transform(train)
scaled_val = targetScaler.transform(val)

exoScaler = Scaler() 
scaled_train_exo = exoScaler.fit_transform(train_exo)
scaled_val_exo = exoScaler.transform(val_exo)

In [ ]:
scaled_target.plot(label="train")
scaled_val.plot(label="val")

Basic model

In [ ]:
early_stopper = EarlyStopping(
    monitor='val_loss',
    patience=10,
    min_delta=1e-3,
    mode='min'
)
lr_scheduler_kwargs = {
    "gamma": 0.999,
}
model = TiDEModel(
    input_chunk_length=48,
    output_chunk_length=12,
    pl_trainer_kwargs={"callbacks": [early_stopper]},
    optimizer_kwargs={"lr": 1e-3},
    lr_scheduler_cls= lr_scheduler.ExponentialLR,
    lr_scheduler_kwargs = {"gamma": 0.999},
    use_reversible_instance_norm = True
)

In [ ]:
model.fit(series=scaled_target,past_covariates=scaled_train_exo,val_series=scaled_val,val_past_covariates=scaled_val_exo,verbose=True)

Not that good so optuna test

In [ ]:
split_date = pd.Timestamp('2022-12')
op_train , op_val = target_series.split_after(split_date)
op_train_exo , op_val_exo = exogenous_series.split_after(split_date)

In [ ]:
import optuna
from darts.metrics import mse

def objective(trial):
    
    # Define early stopping callback
    early_stopper = EarlyStopping(
        monitor='val_loss',
        patience=10,
        min_delta=1e-3,
        mode='min',
        verbose=False
    )

    trial_params = {
        'input_chunk_length': trial.suggest_int("input_chunk_length", 24, 60),
        'output_chunk_length': 12,
        'num_encoder_layers': trial.suggest_int("num_encoder_layers", 1, 3),
        'num_decoder_layers': trial.suggest_int("num_decoder_layers", 1, 3),
        'hidden_size': trial.suggest_int("hidden_size", 64, 512),
        'dropout': trial.suggest_float("dropout", 0.1, 0.5),
        'optimizer_kwargs': {"lr": trial.suggest_float("optimizer_lr", 1e-4, 1e-2, log=False)},
        'lr_scheduler_cls': lr_scheduler.ExponentialLR,
        'lr_scheduler_kwargs': {"gamma": trial.suggest_float("scheduler_gamma", 0.9, 1.0)},
        'pl_trainer_kwargs': {"callbacks": [early_stopper]},
        'use_reversible_instance_norm': trial.suggest_categorical('use_reversible_instance_norm', [True, False]),
    }

    # Initialize the TiDEModel with suggested hyperparameters
    model = TiDEModel(**trial_params)

    # Fit the model
    model.fit(series=scaled_target,past_covariates=scaled_train_exo, val_series=scaled_val, val_past_covariates=scaled_val_exo, verbose=False)

    # Evaluate the model
    val_predictions = model.predict(n=12,series=op_train,past_covariates=op_train_exo, verbose=False)
    transformed_predictions = targetScaler.inverse_transform(val_predictions, verbose=False)
    error = mse(op_val, transformed_predictions, verbose=False)
    return error


In [ ]:
# Create an Optuna study and optimize
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

In [ ]:
# Retrieve the best hyperparameters
best_params = study.best_params
print(f"Best hyperparameters: {best_params}")

Best params made into a model here

In [ ]:
# Initialize the TiDEModel with (now fossilised) suggested hyperparameters
    # Define early stopping callback
#early_stopper = EarlyStopping(
#    monitor='val_loss',
#    patience=10,
#    min_delta=1e-3,
#    mode='min'
#)
#model = TiDEModel(
#    input_chunk_length=29,
#    output_chunk_length=12,
#    num_encoder_layers=3,
#    num_decoder_layers=2,
#    hidden_size=443,
#    dropout= 0.19411763114257125,
#    optimizer_kwargs={"lr": 0.00014544898516544107},
#    lr_scheduler_cls=torch.optim.lr_scheduler.ExponentialLR,
#    lr_scheduler_kwargs={"gamma": 0.9645025339005199},
#    pl_trainer_kwargs={"callbacks": [early_stopper]},
#)

In [ ]:
# make a clean set of parameters from the suggested best parameters
import copy

# a shallow copy will do
clean_params = copy.copy(best_params)
del clean_params['optimizer_lr'], clean_params['scheduler_gamma']

In [ ]:
early_stopper = EarlyStopping(
    monitor='val_loss',
    patience=10,
    min_delta=1e-3,
    mode='min'
)
clean_params['output_chunk_length'] = 12
clean_params['lr_scheduler_cls'] = lr_scheduler.ExponentialLR
clean_params['optimizer_kwargs'] = {'lr': best_params['optimizer_lr']}
clean_params['lr_scheduler_kwargs'] = {'gamma': best_params['scheduler_gamma']}
clean_params['pl_trainer_kwargs'] = {"callbacks": [early_stopper]}

In [ ]:
# Initialize the TiDEModel with suggested hyperparameters
    # Define early stopping callback
model = TiDEModel(**clean_params)

In [ ]:
# Fit the model
model.fit(series=scaled_target,past_covariates=scaled_train_exo, val_series=scaled_val, val_past_covariates=scaled_val_exo)

In [ ]:
# Predict over the test horizon
TEST_HORIZON = 12
val_predictions = model.predict(n=TEST_HORIZON)
transformed_predictions = targetScaler.inverse_transform(val_predictions)

In [ ]:
from Evaluation.Helper.evaluation_helpers import display_results

actuals = val[:TEST_HORIZON].values()
preds = transformed_predictions.values()

display_results(actuals, preds, val[:TEST_HORIZON].time_index, 'TiDE')

In [ ]:
from Evaluation.Helper.evaluation_helpers import calc_metrics_arrays

# Evaluate the model
calc_metrics_arrays(actuals, preds, model_names=['TiDE'])

In [ ]:
model.save("tide.pkl")

Go from here if wanting to load model

In [ ]:
from darts.models import TiDEModel

# Load the model from the file
model = TiDEModel.load("tide.pkl")

In [ ]:
scaled_total = targetScaler.fit_transform(target_series)

scaled_total_exo = exoScaler.fit_transform(exogenous_series)

pred = model.predict(12,series=scaled_total,past_covariates=scaled_total_exo)
finalout = targetScaler.inverse_transform(pred)
print(f'Final predictions:\n{finalout.to_dataframe()}')

In [ ]:
import numpy as np
np.save(os.path.join(project_root, 'Predictions', 'Tide.npy'), finalout.to_dataframe().to_numpy().flatten())